# Estimativa das proporções

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

No capítulo anterior resolvemos o problema das 101 tigelas, e admiti que não se trata realmente de adivinhar de que tigela vieram as bolachas; trata-se de estimar proporções.

Neste capítulo, damos mais um passo em direcção às estatísticas Bayesianas, resolvendo o problema do Euro.

Vamos começar com a mesma distribuição prévia, e veremos que a actualização é a mesma, matematicamente.

Mas argumentarei que é um problema diferente, filosoficamente, e usá-lo-ei para introduzir dois elementos definidores das estatísticas Bayesianas: escolher distribuições anteriores, e usar a probabilidade para representar o desconhecido.

## O problema do Euro

Em *Theory Information, Inference, and Learning Algorithms*, David MacKay coloca este problema:

"Uma declaração estatística apareceu em *The Guardian* na sexta-feira 4 de Janeiro de 2002:

> Quando rodada no bordo 250 vezes, uma moeda belga de um euro veio à cabeça 140 vezes e cauda 110.  \Parece-me muito suspeito", disse Barry Blight, um professor de estatística da London School of Economics.  \Se a moeda fosse imparcial, a hipótese de obter um resultado tão extremo como esse seria inferior a 7%".

"Mas [MacKay pergunta] será que estes dados dão provas de que a moeda é tendenciosa em vez de justa?"

Para responder a essa pergunta, procederemos em duas etapas.

Primeiro usaremos a distribuição binomial para ver de onde vieram esses 7%; depois usaremos o Teorema de Bayes para estimar a probabilidade de esta moeda subir à cabeça.



## A distribuição binomial

Suponhamos que vos digo que uma moeda é "justa", ou seja, a probabilidade de cabeças é de 50%.  Se a rodarmos duas vezes, há quatro resultados: `HH`, `HT`, `TH`, e `TTT`.  Todos os quatro resultados têm a mesma probabilidade, 25%.

Se somarmos o número total de cabeças, há três resultados possíveis: 0, 1, ou 2.  As probabilidades de 0 e 2 são de 25%, e a probabilidade de 1 é de 50%.

Mais genericamente, suponha que a probabilidade de cabeças é de $p$ e nós giramos a moeda $n$ vezes.  A probabilidade de obtermos um total de $k$ cabeças é dada pelo xx_markdown_link_link_xx:

$$$binom{n}{k} p^k (1-p)^{n-k}$$

por qualquer valor de $k$ de 0 a $n$, incluindo ambos.

O termo $\binom{n}{k}$ é o xx_markdown_link_link_xx, normalmente pronunciado "n choose k".

Poderíamos avaliar esta expressão nós próprios, mas também podemos utilizar a função SciPy `binom.pmf`.

Por exemplo, se virarmos uma moeda `n=2` vezes e a probabilidade de cabeças for `p=0,5`, aqui está a probabilidade de obter `k=1` cabeças:

In [4]:
from scipy.stats import binom

n = 2
p = 0.5
k = 1

binom.pmf(k, n, p)

Em vez de fornecer um único valor para "k", também podemos chamar "binom.pmf" com um conjunto de valores.

In [5]:
import numpy as np
ks = np.arange(n+1)

ps = binom.pmf(ks, n, p)
ps

O resultado é uma matriz NumPy com a probabilidade de 0, 1, ou 2 cabeças.

Se colocarmos estas probabilidades num `Pmf`, o resultado é a distribuição de `k` para os valores dados de `n` e `p`.

Aqui está o que parece:

In [6]:
from empiricaldist import Pmf

pmf_k = Pmf(ps, ks)
pmf_k

A função seguinte calcula a distribuição binomial para determinados valores de `n` e `p` e retorna um `Pmf` que representa o resultado.

In [7]:
def make_binomial(n, p):
    """Make a binomial Pmf."""
    ks = np.arange(n+1)
    ps = binom.pmf(ks, n, p)
    return Pmf(ps, ks)

Eis o que parece com `n=250` e `p=0,5`:

In [8]:
pmf_k = make_binomial(n=250, p=0.5)

In [9]:
from utils import decorate

pmf_k.plot(label='n=250, p=0.5')

decorate(xlabel='Number of heads (k)',
         ylabel='PMF',
         title='Binomial distribution')

A quantidade mais provável nesta distribuição é 125:

In [10]:
pmf_k.max_prob()

Mas apesar de ser a quantidade mais provável, a probabilidade de obtermos exactamente 125 cabeças é de apenas cerca de 5%.

In [11]:
pmf_k[125]

No exemplo do MacKay, temos 140 cabeças, o que é ainda menos provável do que 125:

In [12]:
pmf_k[140]

No artigo MacKay cita, o estatístico diz: "Se a moeda fosse imparcial, a hipótese de obter um resultado tão extremo como esse seria inferior a 7%".

Podemos utilizar a distribuição binomial para verificar a sua matemática.  A seguinte função toma um PMF e calcula a probabilidade total de quantidades maiores ou iguais a "limiar". 

In [13]:
def prob_ge(pmf, threshold):
    """Probability of quantities greater than threshold."""
    ge = (pmf.qs >= threshold)
    total = pmf[ge].sum()
    return total

Aqui está a probabilidade de obter 140 cabeças ou mais:

In [14]:
prob_ge(pmf_k, 140)

O `Pmf` fornece um método que faz o mesmo cálculo.

In [15]:
pmf_k.prob_ge(140)

O resultado é cerca de 3,3%, o que é menos do que os 7% citados.  A razão para a diferença é que o estatístico inclui todos os resultados "tão extremos como" 140, o que inclui resultados inferiores ou iguais a 110.

Para ver de onde isso vem, recorde-se que o número esperado de cabeças é de 125.  Se obtivermos 140, teremos excedido essa expectativa em 15.

E se obtivermos 110, temos um défice de 15.

7% é a soma de ambas as "caudas", como se pode ver na figura seguinte.

In [16]:
import matplotlib.pyplot as plt

def fill_below(pmf):
    qs = pmf.index
    ps = pmf.values
    plt.fill_between(qs, ps, 0, color='C5', alpha=0.4)

qs = pmf_k.index
fill_below(pmf_k[qs>=140])
fill_below(pmf_k[qs<=110])
pmf_k.plot(label='n=250, p=0.5')

decorate(xlabel='Number of heads (k)',
         ylabel='PMF',
         title='Binomial distribution')

Eis como calculamos a probabilidade total da cauda esquerda.

In [17]:
pmf_k.prob_le(110)

A probabilidade de resultados inferiores ou iguais a 110 é também de 3,3%,

por isso a probabilidade total de resultados "tão extremos" como 140 é de 6,6%.

O objectivo deste cálculo é que estes resultados extremos são improváveis se a moeda for justa.

Isso é interessante, mas não responde à pergunta de MacKay.  Vamos ver se conseguimos.

## Estimativa Bayesiana

Qualquer moeda tem alguma probabilidade de pousar com a cabeça para cima quando rodada

no limite; chamarei a esta probabilidade `x`.

Parece razoável acreditar que o `x` depende

sobre as características físicas da moeda, como a sua distribuição

de peso.

Se uma moeda estiver perfeitamente equilibrada, esperamos que `x` esteja perto de 50%, mas

por uma moeda distorcida, `x` pode ser substancialmente diferente.

Podemos utilizar o teorema de Bayes e os dados observados para estimar `x`.

Para simplificar, vou começar com um anterior uniforme, que pressupõe que todos os valores de `x` são igualmente prováveis.

Isso pode não ser uma suposição razoável, por isso voltaremos mais tarde e consideraremos outros antecedentes.

Podemos fazer um prior uniforme como este:

In [18]:
hypos = np.linspace(0, 1, 101)
prior = Pmf(1, hypos)

O "hypos" é um conjunto de valores igualmente espaçados entre 0 e 1.

Podemos utilizar as hipóteses para calcular as probabilidades, desta forma:

In [19]:
likelihood_heads = hypos
likelihood_tails = 1 - hypos

Vou colocar as probabilidades de cabeças e caudas num dicionário para facilitar a actualização.

In [20]:
likelihood = {
    'H': likelihood_heads,
    'T': likelihood_tails
}

Para representar os dados, vou construir um cordel com "H" repetido 140 vezes e "T" repetido 110 vezes.

In [21]:
dataset = 'H' * 140 + 'T' * 110

A função seguinte faz a actualização.

In [22]:
def update_euro(pmf, dataset):
    """Update pmf with a given sequence of H and T."""
    for data in dataset:
        pmf *= likelihood[data]

    pmf.normalize()

O primeiro argumento é um `Pmf` que representa o anterior.

O segundo argumento é uma sequência de cordas.

De cada vez, multiplicamos `pmf' pela probabilidade de um resultado, `H' para cabeças ou `T' para caudas.

Note-se que "normalizar" está fora do laço, pelo que a distribuição posterior só se normaliza uma vez, no final.

Isso é mais eficiente do que normalizá-lo após cada centrifugação (embora veremos mais tarde que também pode causar problemas com a aritmética de ponto flutuante).

Eis como utilizamos `update_euro`.

In [23]:
posterior = prior.copy()
update_euro(posterior, dataset)

E aqui está o aspecto do posterior.

In [24]:
def decorate_euro(title):
    decorate(xlabel='Proportion of heads (x)',
             ylabel='Probability',
             title=title)

In [25]:
posterior.plot(label='140 heads out of 250', color='C4')
decorate_euro(title='Posterior distribution of x')

Esta figura mostra a distribuição posterior de `x`, que é a proporção de cabeças para a moeda que observámos.

A distribuição posterior representa as nossas crenças sobre o `x` depois de ver os dados.

Indica que valores inferiores a 0,4 e superiores a 0,7 são improváveis; os valores entre 0,5 e 0,6 são os mais prováveis.

De facto, o valor mais provável para `x` é 0,56 que é a proporção de cabeças no conjunto de dados, `140/250`.

In [26]:
posterior.max_prob()

## Triângulo Prior

Até agora temos estado a usar um uniforme anterior:

In [27]:
uniform = Pmf(1, hypos, name='uniform')
uniform.normalize()

Mas isso pode não ser uma escolha razoável com base no que sabemos sobre moedas.

Posso acreditar que se uma moeda for desequilibrada, `x` pode desviar-se substancialmente de 0,5, mas parece improvável que a moeda de Euro belga seja tão desequilibrada que `x` seja 0,1 ou 0,9.

Talvez seja mais razoável escolher um anterior que dê

maior probabilidade para valores de `x` próximos de 0,5 e menor probabilidade

a valores extremos.

Como exemplo, vamos tentar um triângulo anterior em forma de triângulo.

Aqui está o código que o constrói:

In [28]:
ramp_up = np.arange(50)
ramp_down = np.arange(50, -1, -1)

a = np.append(ramp_up, ramp_down)

triangle = Pmf(a, hypos, name='triangle')
triangle.normalize()

A "gama" devolve uma matriz NumPy, para que possamos utilizar `np.append' para anexar `ramp_down' ao fim de `ramp_up'.

Depois utilizamos `a' e `hypos' para fazer um `Pmf'.

A figura seguinte mostra o resultado, juntamente com o anterior uniforme.

In [29]:
uniform.plot()
triangle.plot()
decorate_euro(title='Uniform and triangle prior distributions')

Agora podemos actualizar ambos os antecedentes com os mesmos dados:



In [30]:
update_euro(uniform, dataset)
update_euro(triangle, dataset)

Aqui estão os posteriors.

In [31]:
uniform.plot()
triangle.plot()
decorate_euro(title='Posterior distributions')

As diferenças entre as distribuições posteriores são pouco visíveis, e tão pequenas que dificilmente importariam na prática.

E isso são boas notícias.

Para ver porquê, imagine duas pessoas que discordam raivosamente sobre qual o anterior é melhor, uniforme ou triangular.

Cada um deles tem razões para a sua preferência, mas nenhum deles consegue persuadir o outro a mudar de ideias.

Mas suponhamos que concordam em utilizar os dados para actualizar as suas crenças.

Quando comparam as suas distribuições posteriores, descobrem que já não há quase nada para discutir.

Este é um exemplo de **acumulando os antecedentes**: com suficiente

dados, as pessoas que começam com antecedentes diferentes tenderão a

convergem para a mesma distribuição posterior.



## A função Binomial da Probabilidade

Até agora temos computado as actualizações uma vez de cada vez, por isso para o problema do Euro temos de fazer 250 actualizações.

Uma alternativa mais eficiente é calcular de uma só vez a probabilidade de todo o conjunto de dados.

Para cada valor hipotético de `x`, temos de calcular a probabilidade de obter 140 cabeças em 250 rotações.

Bem, sabemos como fazer isso; esta é a questão que o binómio de distribuição responde.

Se a probabilidade de cabeças é de $p$, a probabilidade de cabeças de $k$ em $n$ giros é de $n$:

$$$binom{n}{k} p^k (1-p)^{n-k}$$

E podemos utilizar a SciPy para o calcular.

A função que se segue toma um `Pmf` que representa uma distribuição prévia e um tuple de inteiros que representam os dados:

In [32]:
from scipy.stats import binom

def update_binomial(pmf, data):
    """Update pmf using the binomial distribution."""
    k, n = data
    xs = pmf.qs
    likelihood = binom.pmf(k, n, xs)
    pmf *= likelihood
    pmf.normalize()

Os dados são representados com um conjunto de valores para `k` e `n`, em vez de um longo conjunto de resultados.

Aqui está a actualização.

In [33]:
uniform2 = Pmf(1, hypos, name='uniform2')
data = 140, 250
update_binomial(uniform2, data)

E aqui está o aspecto do posterior.

In [34]:
uniform.plot()
uniform2.plot()
decorate_euro(title='Posterior distributions computed two ways')

Podemos utilizar `allclose` para confirmar que o resultado é o mesmo que na secção anterior, excepto para um pequeno arredondamento de vírgula flutuante.

In [35]:
np.allclose(uniform, uniform2)

Mas esta forma de fazer o cálculo é muito mais eficiente.

## Estatísticas Bayesianas

Talvez tenha notado semelhanças entre o problema do Euro e o problema das 101 Taças emÂ <<_101Bowls>>>>.

As distribuições anteriores são as mesmas, as probabilidades são as mesmas, e com os mesmos dados os resultados seriam os mesmos.

Mas há duas diferenças.

A primeira é a escolha do prior.

Com 101 tigelas, o anterior uniforme está implícito na declaração do problema, que diz que escolhemos uma das tigelas ao acaso com igual probabilidade.

No problema do Euro, a escolha do prior é subjectiva; ou seja, pessoas razoáveis poderiam discordar, talvez porque têm informações diferentes sobre moedas ou porque interpretam a mesma informação de forma diferente.

Como os antecedentes são subjectivos, os posteriors também são subjectivos.

E algumas pessoas acham isso problemático.

A outra diferença é a natureza do que estamos a estimar.

No problema das 101 Taças, escolhemos a taça ao acaso, pelo que é incontroverso calcular a probabilidade de escolher cada taça.

No problema do Euro, a proporção de cabeças é uma propriedade física de uma dada moeda.

Sob algumas interpretações de probabilidade, isso é um problema porque as propriedades físicas não são consideradas aleatórias.

Como exemplo, considere a idade do universo.

Actualmente, a nossa melhor estimativa é de 13,80 mil milhões de anos, mas pode estar fora por 0,02 mil milhões de anos em qualquer direcção (ver [here](https://en.wikipedia.org/wiki/Age_of_the_universe)).

Agora suponhamos que gostaríamos de saber a probabilidade de a idade do universo ser realmente superior a 13,81 mil milhões de anos.

Sob algumas interpretações de probabilidade, não seríamos capazes de responder a essa pergunta.

Seríamos obrigados a dizer algo como, "A idade do universo não é uma quantidade aleatória, pelo que não tem probabilidade de exceder um determinado valor".

Sob a interpretação Bayesiana da probabilidade, é significativo e útil tratar quantidades físicas como se fossem aleatórias e calcular probabilidades sobre elas.

No problema do Euro, a distribuição prévia representa o que acreditamos sobre as moedas em geral e a distribuição posterior representa o que acreditamos sobre uma determinada moeda depois de vermos os dados.

Assim, podemos utilizar a distribuição posterior para calcular probabilidades sobre a moeda e a sua proporção de cabeças.

A subjectividade do anterior e a interpretação do posterior são diferenças chave entre a utilização do Teorema de Bayes e a realização de estatísticas Bayesianas.

O Teorema de Bayes é uma lei matemática de probabilidade; nenhuma pessoa razoável se opõe a ele.

Mas as estatísticas Bayesianas são surpreendentemente controversas.

Historicamente, muitas pessoas têm sido incomodadas pela sua subjectividade e pelo seu uso da probabilidade para coisas que não são aleatórias.

Se estiver interessado nesta história, recomendo o livro de Sharon Bertsch McGrayne, *[The Theory That Would Not Die](https://yalebooks.yale.edu/book/9780300188226/theory-would-not-die)**.

## Resumo

Neste capítulo coloquei o problema europeu de David MacKay e começámos a resolvê-lo.

Dados os dados, calculámos a distribuição posterior para `x`, a probabilidade de uma moeda de Euro subir à cabeça.

Tentámos dois antecedentes diferentes, actualizámo-los com os mesmos dados, e descobrimos que os posteriors eram quase os mesmos.

Esta é uma boa notícia, porque sugere que se duas pessoas começam com crenças diferentes e vêem os mesmos dados, as suas crenças tendem a convergir.

Este capítulo introduz a distribuição binomial, que utilizámos para calcular a distribuição posterior de forma mais eficiente.

E discuti as diferenças entre aplicar o Teorema de Bayes, como no problema das 101 Taças, e fazer estatísticas Bayesianas, como no problema do Euro.

No entanto, ainda não respondemos à pergunta de MacKay: "Será que estes dados dão provas de que a moeda é tendenciosa e não justa?"

Vou deixar esta pergunta pendurada um pouco mais; voltaremos a ela em <<_Testing>>>>.

No próximo capítulo, vamos resolver problemas relacionados com a contagem, incluindo comboios, tanques e coelhos.

Mas primeiro talvez queira trabalhar nestes exercícios.

## Exercícios



**Exercício:** Na Liga Principal de Basebol, a maioria dos jogadores tem uma média de rebatidas entre 0,200 e 0,330, o que significa que a sua probabilidade de obter uma pancada está entre 0,2 e 0,33.

Suponha que um jogador que apareça no seu primeiro jogo recebe 3 pancadas em 3 tentativas.  Qual é a distribuição posterior para a sua probabilidade de obter um acerto?

Para este exercício, vou construir a distribuição prévia começando com uma distribuição uniforme e actualizando-a com dados imaginários até que tenha uma forma que reflicta o meu conhecimento de base das médias de batimentos.

Aqui está o anterior uniforme:

In [36]:
hypos = np.linspace(0.1, 0.4, 101)
prior = Pmf(1, hypos)

E aqui está um dicionário de probabilidades, com `Y` por obter um acerto e `N` por não obter um acerto.

In [37]:
likelihood = {
    'Y': hypos,
    'N': 1-hypos
}

Aqui está um conjunto de dados que produz uma distribuição prévia razoável.

In [38]:
dataset = 'Y' * 25 + 'N' * 75

E aqui está a actualização com os dados imaginários.

In [39]:
for data in dataset:
    prior *= likelihood[data]

prior.normalize()

Finalmente, eis como se parece o anterior.

In [40]:
prior.plot(label='prior')
decorate(xlabel='Probability of getting a hit',
         ylabel='PMF')


Agora actualize esta distribuição com os dados e trace a posterior.  Qual é a quantidade mais provável na distribuição posterior?

In [41]:
# Solution goes here

In [42]:
# Solution goes here

In [43]:
# Solution goes here

In [44]:
# Solution goes here

**Exercício:** Sempre que se faz um inquérito sobre questões sensíveis, é preciso lidar com [social desirability bias](https://en.wikipedia.org/wiki/Social_desirability_bias), que é a tendência das pessoas para ajustar as suas respostas de modo a mostrarem-se na luz mais positiva.

Uma forma de melhorar a precisão dos resultados é [randomized response](https://en.wikipedia.org/wiki/Randomized_response).

Como exemplo, suponha que quer saber quantas pessoas enganam os seus impostos.

Se lhes perguntar directamente, é provável que alguns dos batoteiros mintam.

Pode obter uma estimativa mais precisa se lhes pedir indirectamente, desta forma: Peça a cada pessoa que atire uma moeda ao ar e, sem revelar o resultado,

* Se receberem cabeças, reportam SIM.

* Se receberem caudas, respondem honestamente à pergunta "Fazem batota com os vossos impostos?

Se alguém disser SIM, não sabemos se realmente enganam os seus impostos; podem ter virado as cabeças.

Sabendo isto, as pessoas podem estar mais dispostas a responder honestamente.

Suponha-se que faz um inquérito a 100 pessoas desta forma e obtém 80 SIM e 20 NÃO.  Com base nestes dados, qual é a distribuição posterior para a fracção de pessoas que fazem batota nos seus impostos?  Qual é a quantidade mais provável na distribuição posterior?

In [45]:
# Solution goes here

In [46]:
# Solution goes here

In [47]:
# Solution goes here

In [48]:
# Solution goes here

In [49]:
# Solution goes here

**Exercício:** Suponha que quer testar se uma moeda é justa, mas não quer girá-la centenas de vezes.

Assim, faz-se uma máquina que gira a moeda automaticamente e usa a visão por computador para determinar o resultado.

No entanto, descobre-se que a máquina nem sempre é precisa.  Especificamente, suponha que a probabilidade é `y=0,2` de uma cabeça real ser reportada como cauda, ou caudas reais reportadas como cabeças.

Se girarmos uma moeda 250 vezes e a máquina informar 140 cabeças, qual é a distribuição posterior de `x`?

O que acontece quando se varia o valor de `y`?

In [50]:
# Solution goes here

In [51]:
# Solution goes here

In [52]:
# Solution goes here

In [53]:
# Solution goes here

**Exercício:** Em preparação para uma invasão alienígena, a Liga de Defesa da Terra (EDL) tem estado a trabalhar em novos mísseis para abater os invasores espaciais.  Claro que alguns desenhos de mísseis são melhores do que outros; vamos assumir que cada desenho tem alguma probabilidade de atingir uma nave alienígena, `x`.

Com base em testes anteriores, a distribuição de `x` na população de desenhos é aproximadamente uniforme entre 0,1 e 0,4.

Agora suponha-se que o novo Extraterrestre ultra-secreto Blaster 9000 está a ser testado.  Numa conferência de imprensa, uma EDL relata que o novo desenho foi testado duas vezes, tirando duas fotografias durante cada teste.  Os resultados do teste são confidenciais, pelo que o general não diz quantos alvos foram atingidos, mas informa: "O mesmo número de alvos foi atingido nos dois testes, por isso temos razões para pensar que esta nova concepção é consistente".

Estes dados são bons ou maus? 

Ou seja, aumenta ou diminui a sua estimativa de `x` para o Alien Blaster 9000?

Dica: Se a probabilidade de acertar cada alvo for $x$, a probabilidade de acertar um alvo em ambos os testes



In [54]:
# Solution goes here

In [55]:
# Solution goes here

In [56]:
# Solution goes here

In [57]:
# Solution goes here

In [58]:
# Solution goes here

In [59]:
# Solution goes here

In [60]:
# Solution goes here